In [2]:
import autograd.numpy as np
from autograd import grad, jacobian, hessian
from autograd.scipy.stats import norm
import time
from scipy.optimize import minimize
from scipy.optimize import fsolve
from autograd import grad
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

---

In [ ]:
def objective(X):
    x, y, z = X
    return x**2 + y**2 + z**2

In [ ]:
def eq(X):
    x, y, z = X
    return 2 * x - y + z - 3

In [ ]:
sol = minimize(objective, [1, -0.5, 0.5], constraints={'type': 'eq', 'fun': eq})
sol

---

In [ ]:
def F(L):
    'Augmented Lagrange function'
    x, y, z, _lambda = L
    return objective([x, y, z]) - _lambda * eq([x, y, z])

In [ ]:
# Gradients of the Lagrange function
dfdL = grad(F, 0)

In [ ]:
# Find L that returns all zeros in this function.
def obj(L):
    x, y, z, _lambda = L
    dFdx, dFdy, dFdz, dFdlam = dfdL(L)
    return [dFdx, dFdy, dFdz, eq([x, y, z])]

In [ ]:
x, y, z, _lam = fsolve(obj, [0.0, 0.0, 0.0, 1.0])
print(f'The answer is at {x, y, z}')

---

In [ ]:
X = np.array([[]])
m = X.shape[0]
Y = []
t1 = z1 = np.array([[ 0.00043896], [ 0.00058852], [-0.00043738], [ 0.00101669]])
t2 = z2 = np.array([[ 0.00119033], [ 0.00065769], [-0.00014218], [ 0.0018238 ]])
t3 = z3 = np.array([[ 0.00029429], [ 0.0004789 ], [-0.00032502], [ 0.00110983]])

Theta = {'w1': t1, 'w2': t2, 'w3': t3}
Zed = {'w1': z1, 'w2': z2, 'w3': z3}
AA = {'w1': 0, 'w2': 0, 'w3': 0}

In [ ]:
def F(L):
    'Augmented Lagrange function'
    T, Z, A = L
    
    q = np.sum(np.array([np.linalg.norm(T - Theta[j])**2 for j in Theta]), axis=0)
    a = np.sum( A * (T - Z) +  np.array([np.linalg.norm(T - Theta[j])**2 for j in Theta]), axis=0)
    return objective([x, y, z]) - _lambda * eq([x, y, z])


np.sum(np.array([w * np.linalg.norm(T[i] - T[j]) ** 2 for j, w in node.W.items()]), axis=0)



sigma_Q = 
            sigma_A = np.sum(np.array([node.A[i] + node.rho * (T[i] - node.Z[i]) for _ in node.Theta]), axis=0)
            cl_dw = sigma_Q + node.mu * node.D * dw + sigma_A

---

In [ ]:
import numpy as np
import scipy as sp
from scipy import optimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import joblib
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# ============================
# Minimizing the Rosenbrock Function
# ============================
def rosenbrock(x):    
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 1.8, 2.9, 1.8])
res = optimize.minimize(rosenbrock, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
res.x

In [ ]:
# ============================
# Function to compute gradient
# ============================
def rosen_grad(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

res = optimize.minimize(rosenbrock, x0, method='BFGS', jac=rosen_grad, options={'disp': True})
res.x

In [ ]:
# Defining data generator
def generate_data(t, A, sigma, omega, noise=0, n_outliers=0, random_state=0):
    y = A * np.exp(-sigma * t) * np.sin(omega * t)
    rnd = np.random.RandomState(random_state)
    error = noise * rnd.randn(t.size)
    outliers = rnd.randint(0, t.size, n_outliers)
    error[outliers] *= 35
    return y + error

# Model parameters
A = 2
sigma = 0.1
omega = 0.1 * 2 * np.pi
x_true = np.array([A, sigma, omega])
noise = 0.1
t_min = 0
t_max = 30

# Outliers
t_train = np.linspace(t_min, t_max, 30)
y_train = generate_data(t_train, A, sigma, omega, noise=noise, n_outliers=4)

# Function to compute residuals of least-squares
def fun(x, t, y):
    return x[0] * np.exp(-x[1] * t) * np.sin(x[2] * t) - y

# Initial estimate
x0 = np.ones(3)

# Running standard least-squares
res_lsq = optimize.least_squares(fun, x0, args=(t_train, y_train))

# Running robust least-squares
res_robust = optimize.least_squares(fun, x0, loss='soft_l1', f_scale=0.1, args=(t_train, y_train))

# Visualization
t_test = np.linspace(t_min, t_max, 300)
y_test = generate_data(t_test, A, sigma, omega)
y_lsq = generate_data(t_test, *res_lsq.x)
y_robust = generate_data(t_test, *res_robust.x)
plt.figure(figsize=(8, 5))
plt.plot(t_train, y_train, 'o', label='data')
plt.plot(t_test, y_test, label='true')
plt.plot(t_test, y_lsq, label='lsq')
plt.plot(t_test, y_robust, label='robust lsq')
plt.xlabel('$t$')
plt.ylabel('$y$')
plt.legend()
plt.show()

In [4]:
import numpy as np
import scipy as sp
from scipy import optimize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import joblib
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [5]:
dataset = "./datasets/mnist.data"
X, y = joblib.load(dataset)
y = y.astype(np.int)

In [6]:
x_train = X[:500]
y_train = y[:500]
x_test = X[60000:]
y_test = y[60000:]
y_train = np.squeeze(y_train)
x_train = x_train[np.any([y_train == 1, y_train == 2], axis = 0)]
y_train = y_train[np.any([y_train == 1, y_train == 2], axis = 0)]
y_train = y_train - 1
y_train = y_train.reshape(-1, 1)
y_test = np.squeeze(y_test)
x_test = x_test[np.any([y_test == 1, y_test == 2], axis = 0)]
y_test = y_test[np.any([y_test == 1, y_test == 2], axis = 0)]
y_test = y_test - 1
y_test = y_test.reshape(-1, 1)
x_train = x_train / 255
x_test = x_test / 255
m = x_train.shape[0]
m_test = x_test.shape[0]
x_train, x_test = x_train.T, x_test.T
y_train, y_test = y_train.reshape(1,m), y_test.reshape(1,m_test)
np.random.seed(138)
shuffle_index = np.random.permutation(m)
X_train, y_train = x_train[:,shuffle_index], y_train[:,shuffle_index]

In [7]:
X_train.shape, y_train.shape

((784, 118), (1, 118))

In [ ]:
def sigmoid(z):
    s = 1.0 / (1.0 + np.exp(-z))
    return s
def compute_loss(Y, Y_hat):

    m = Y.shape[1]
    L = -(1./m) * ( np.sum( np.multiply(np.log(Y_hat),Y) ) + np.sum( np.multiply(np.log(1-Y_hat),(1-Y)) ) )

    return L

learning_rate = 0.05

X = X_train
Y = y_train

n_x = X.shape[0]
m = X.shape[1]

W = np.random.randn(n_x, 1) * 0.01
b = np.zeros((1, 1))

for i in range(2000):
    Z = np.matmul(W.T, X) + b
    A = sigmoid(Z)

    cost = compute_loss(Y, A)

    dW = (1/m) * np.matmul(X, (A-Y).T)
    db = (1/m) * np.sum(A-Y, axis=1, keepdims=True)

    W = W - learning_rate * dW
    b = b - learning_rate * db

    if (i % 200 == 0):
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)

Z = np.matmul(W.T, x_test) + b
A = sigmoid(Z)

predictions = (A>.5)[0,:]

print(predictions.shape, A.shape)
labels = (y_test == 1)[0,:]

accuracy_score(predictions, labels)

In [ ]:
#models = []

In [ ]:
models.append({"W": W, "b": b})

In [ ]:
len(models)

In [ ]:
#joblib.dump(models, 'models.data')

In [ ]:
models = joblib.load('./models.data')

In [ ]:
Z = models

In [ ]:
np.linalg.norm(models[2]["W"] - models[1]["W"])

In [ ]:
np.sqrt(np.sum((models[2]["W"] - models[1]["W"])**2))

In [ ]:
def obj(theta, i=1):
    m = X.shape[1]

    z = np.dot(theta.T, X_train) + models[i]["b"]

    A = sigmoid(z)

    cost = -1.0/m * np.sum(y_train * np.log(A) + (1.0 - y_train) * np.log(1.0 - A))

#     sigmaQ = np.sum(np.array([0.6 * (np.sqrt(np.sum((theta - model["W"])**2))) ** 2 for model in models]), axis=0)
    
#     Q = 0.5 * sigmaQ * 0.5 * 2 * cost
    
    print(cost)
    
    return cost

In [ ]:
i = 1
jacobian_ = jacobian(obj)
theta_start  = models[i]["W"]
b_start  = models[i]["b"]

In [ ]:
obj(theta_start)

In [ ]:
jacobian_(obj)

In [ ]:
jacobian_(theta_start)

In [ ]:
jacobian_(theta_start)

print("Initial loss:", obj(theta_start))
for i in range(1):
    theta_start -= jacobian_(theta_start) * 0.01

print("Trained loss:", obj(theta_start))

In [ ]:
start_time = time.time()

res = optimize.minimize(obj, theta_start, method='BFGS', options={'disp': False}, jac=jacobian_)

print(f"Optimization done in {time.time() - start_time} seconds")
res.x

In [ ]:
obj(a)

In [ ]:
optimize.minimize(rosenbrock, x0, method='BFGS', jac=rosen_grad, options={'disp': True})
res.x

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from builtins import range
#import autograd.numpy as np
import numpy as np
from autograd import grad
from autograd.test_util import check_grads

models = joblib.load('./models.data')
i = 0
theta_start  = models[i]["W"]
m = X_train.shape[1]

def sigmoid(x):
    return 0.5*(np.tanh(x) + 1)

def logistic_predictions(weights, inputs):
    # Outputs probability of a label being true according to logistic model.
    return sigmoid(np.dot(inputs.T, weights))

def training_loss(weights):
    # Training loss is the negative log-likelihood of the training labels.
    preds = logistic_predictions(weights, inputs)
    label_probabilities = preds * targets + (1 - preds) * (1 - targets)
    print(-np.sum(np.log(label_probabilities)))
    return (1/m) * -np.sum(np.log(label_probabilities))

# Build a toy dataset.
inputs = X_train
targets = y_train

# Build a function that returns gradients of training loss using autograd.
training_gradient_fun = grad(training_loss)

# Check the gradients numerically, just to be safe.
weights = np.zeros((X_train.shape[0], 1))
check_grads(training_loss, modes=['rev'])(weights)
print(weights.shape)

# Optimize weights using gradient descent.
print("Initial loss:", training_loss(weights))
for i in range(500):
    weights -= training_gradient_fun(weights) * 0.01

print("Trained loss:", training_loss(weights))

In [ ]:
np.zeros((3, 1))

---

In [11]:
import cvxpy as cp
import numpy as np



models = joblib.load('./models.data')
i = 1
theta_start  = models[i]["W"]
b_start  = models[i]["b"]

def sigmoid(z):
    s = 1.0 / (1.0 + cp.exp(-z))
    return s


def L(theta):
    m = X.shape[1]

    z = np.dot(theta.T, X_train) + models[i]["b"]

    A = sigmoid(z)

    cost = -1.0/m * np.sum(y_train * np.log(A) + (1.0 - y_train) * np.log(1.0 - A))

#     sigmaQ = np.sum(np.array([0.6 * (np.sqrt(np.sum((theta - model["W"])**2))) ** 2 for model in models]), axis=0)
    
#     Q = 0.5 * sigmaQ * 0.5 * 2 * cost
    
    print(cost)
    
    return cost



# Create two scalar optimization variables.
x = cp.Variable(theta_start.shape)
x.value = theta_start

# Form objective.
obj = cp.Minimize(L(x))

# Form and solve problem.
prob = cp.Problem(obj)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

ValueError: setting an array element with a sequence.

In [ ]:
cp.Variable?